In [2]:
import mysql.connector
import random
from faker import Faker
from datetime import datetime

# Initialize Faker for generating random data
fake = Faker()

# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "soil_management"
}

# Database Connection Function
def connect_db():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        return conn
    except mysql.connector.Error as e:
        print(f"Error connecting to database: {e}")
        return None

# Function to Insert Manual Soil Record
def insert_manual_record(farm_location, test_date, nitrogen, phosphorus, potassium, pH, moisture):
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                INSERT INTO soil_health (farm_location, test_date, nitrogen_level, phosphorus_level, potassium_level, pH_level, moisture_content)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (farm_location, test_date, nitrogen, phosphorus, potassium, pH, moisture))
            conn.commit()
            print("Soil record inserted successfully!")
        except mysql.connector.Error as e:
            print(f"Error inserting record: {e}")
        finally:
            cursor.close()
            conn.close()

# Function to Generate Random Data for Bulk Insert
def generate_soil_data():
    farm_location = fake.city()
    test_date = fake.date_between(start_date="-2y", end_date="today")
    nitrogen = round(random.uniform(0.1, 5.0), 2)
    phosphorus = round(random.uniform(0.1, 5.0), 2)
    potassium = round(random.uniform(0.1, 5.0), 2)
    pH = round(random.uniform(4.5, 8.5), 2)
    moisture = round(random.uniform(5.0, 50.0), 2)
    return (farm_location, test_date, nitrogen, phosphorus, potassium, pH, moisture)

# Function to Insert Bulk Records
def insert_bulk_records(total_records, batch_size):
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        for i in range(0, total_records, batch_size):
            data_batch = [generate_soil_data() for _ in range(min(batch_size, total_records - i))]
            cursor.executemany("""
                INSERT INTO soil_health (farm_location, test_date, nitrogen_level, phosphorus_level, potassium_level, pH_level, moisture_content)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, data_batch)
            conn.commit()
        print(f"{total_records} records inserted successfully!")
        cursor.close()
        conn.close()

# Function to Display Records
def display_records(limit=None):
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        query = "SELECT * FROM soil_health ORDER BY record_no DESC"
        if limit:
            query += f" LIMIT {limit}"
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        print("Records:")
        for row in rows:
            print(row)

# Create Table if Not Exists
def create_table():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS soil_health (
                    record_no INT AUTO_INCREMENT PRIMARY KEY,
                    farm_location VARCHAR(255) NOT NULL,
                    test_date DATE,
                    nitrogen_level FLOAT,
                    phosphorus_level FLOAT,
                    potassium_level FLOAT,
                    pH_level FLOAT,
                    moisture_content FLOAT
                );
            """
            )
            conn.commit()
            print("Table created successfully!")
        except mysql.connector.Error as err:
            print(f"Error creating table: {err}")
        finally:
            cursor.close()
            conn.close()

# Example Usage
if __name__ == "__main__":
    create_table()
    insert_manual_record("Pune", "2024-02-24", 2.5, 1.8, 3.0, 6.5, 20.0)
    insert_bulk_records(10, 5)
    display_records(10)


Table created successfully!
Soil record inserted successfully!
10 records inserted successfully!
Records:
(11, 'Rodgersbury', datetime.date(2024, 3, 3), 0.97, 0.18, 4.01, 7.68, 12.92)
(10, 'Ericksonchester', datetime.date(2023, 4, 4), 4.35, 1.33, 4.46, 7.96, 15.65)
(9, 'Hickmanhaven', datetime.date(2023, 7, 30), 4.88, 1.56, 1.88, 6.18, 25.21)
(8, 'Port Cameronmouth', datetime.date(2023, 12, 26), 1.42, 0.35, 3.69, 8.3, 30.93)
(7, 'New Janice', datetime.date(2023, 3, 8), 1.42, 2.95, 2.61, 4.67, 19.91)
(6, 'Nelsonton', datetime.date(2024, 3, 10), 1.84, 2.06, 0.43, 5.08, 39.01)
(5, 'Robinsonhaven', datetime.date(2024, 10, 1), 0.83, 3.1, 2.48, 7.52, 19.93)
(4, 'Osbornemouth', datetime.date(2024, 1, 1), 4.08, 1.83, 4.61, 7.15, 48.45)
(3, 'Murphyshire', datetime.date(2023, 9, 23), 3.27, 0.43, 0.68, 5.51, 37.53)
(2, 'Lake Ethanborough', datetime.date(2023, 12, 7), 3.38, 2.43, 3.05, 6.85, 42.17)
